# Buscador de texto en Wikipedia por palabras claves usando VSM y similitud coseno
### Autores: Miguel Pedraza, Yenner Robayo

In [1]:
import json
import utils
from time import time
from collections import Counter

# Definición de metodos utilizados en el algoritmo de busqueda

def opendDocuments():
    file_inverdIndex = open("inverterIndex.txt", "r", encoding='utf-8')
    inverdIndex = json.JSONDecoder().decode(file_inverdIndex.read())
    file_inverdIndex.close()

    file_stopwords = open("spanish.txt", "r", encoding='utf-8')
    stopwords = file_stopwords.read().split()
    file_stopwords.close()

    file_words = open("tfidf.txt", "r", encoding='utf-8')
    allTfidf = json.JSONDecoder().decode(file_words.read())
    file_words.close()

    file_documents = open("corpus.txt", "r", encoding='utf-8')
    corpus = json.JSONDecoder().decode(file_documents.read())
    file_documents.close()

    file_inv_frec_vector = open("inv_frec_vector.txt", "r", encoding='utf-8')
    inv_frec_vector = json.JSONDecoder().decode(file_inv_frec_vector.read())
    file_inv_frec_vector.close()

    return inverdIndex, stopwords, allTfidf, corpus, inv_frec_vector

def search(query):
    dataset = {}
    tfidf = [0] * len(inverdIndex)
    cos_sim = {}

    doc_listwords = []
    for word in utils.removeSymbols(query.lower()).split():
        if word not in stopwords and utils.isNotEmpty(word):
            doc_listwords.append(word)
    dataset = Counter(doc_listwords)
    
    for id, word in enumerate(inverdIndex.keys()):
        if word in dataset:
            tfidf[id] = dataset.get(word, 0)*inv_frec_vector[id]             

    for word in dataset.keys():
        if word in inverdIndex:  # si la palabra esta en el index invertido
            for key in inverdIndex.get(word):
                if key not in cos_sim:                    
                    cos_sim[key] = utils.cosineSimilarity(tfidf, allTfidf[key])
    return cos_sim


In [2]:
# abrir documentos con los documentos procesados
inverdIndex, stopwords, allTfidf, corpus, inv_frec_vector = opendDocuments()

### Algoritmo de Búsqueda

In [3]:

query = input("Ingrese la consulta: ")

# obtiene el tiempo actual
t0 = time()

# realiza la consulta
docs = search(query)

# obtiene el tiempo total de la busqueda
totalTime = time()-t0

# muestra resultados
print()
print("Tiempo total de la busqueda %0.3fs." % totalTime)
print("Total de documentos encontrados: %d" % len(docs))
print("La consulta es: " + query)
print()

i = 0
for key in sorted(docs, key=docs.get, reverse=True):
    print("Documento encontrado: #%s, cs: %f" % (i, docs[key]))
    print('Documento #%s: %.500s' % (key, corpus[key]))
    print()
    i += 1

    if (i > 10):
        break


Ingrese la consulta: universidad de los llanos

Tiempo total de la busqueda 0.682s.
Total de documentos encontrados: 55
La consulta es: universidad de los llanos

Documento encontrado: #0, cs: 0.040172
Documento #76: José Antonio Balseiro José Antonio Balseiro (Córdoba, 29 de marzo de 1919 - Bariloche, 26 de marzo de 1962) fue un importante físico argentino.
José Antonio Balseiro nació en la ciudad de Córdoba el 29 de marzo de 1919, cuarto hijo de Antonio Balseiro, quien había emigrado de España en su adolescencia, y de Victoria Lahore, argentina de origen francés.
En 1933 ingresa al Colegio Nacional de Monserrat dependiente de la Universidad de Córdoba, de donde egresa con el título de bachiller en 1938.
Ba

Documento encontrado: #1, cs: 0.038633
Documento #8: Comitán de Domínguez Comitán de Domínguez (antes Comitán de las Flores) en honor al prócer de la palabra libre Belisario Domínguez Palencia, el mayor héroe civil de México nacido en Comitán (Komitl-tlan lugar de alfareros) palab